In [1]:
!pip install -q tritonclient[http]
!pip install -q pandas scikit-learn

In [1]:
import triton_python_backend_utils as pb_utils

In [2]:
import tritonclient.http as httpclient
from tritonclient.utils import np_to_triton_dtype

In [3]:
import pandas as pd
import numpy as np
import json

In [4]:
amount = np.array(['$45.23'])

In [5]:
data = pd.DataFrame({'Amount' : amount})
data['Amount'] = data['Amount'].str.slice(1)

In [6]:
data

,Amount
0,45.23


In [7]:
amount = data['Amount'].values

In [8]:
client = httpclient.InferenceServerClient(url='localhost:8000')

In [9]:
def prepare_tensor(name, input_data):
    tensor = httpclient.InferInput(
        name, input_data.shape, np_to_triton_dtype(input_data.dtype))
    tensor.set_data_from_numpy(input_data)
    return tensor

In [10]:
model_name = "preprocessing"

In [33]:
amount_data = np.array(["$34.56"]).astype(str)
inputs = [httpclient.InferInput("AMOUNT", amount_data.shape,
                              np_to_triton_dtype(amount_data.dtype))]

In [35]:
??np_to_triton_dtype

Signature: np_to_triton_dtype(np_dtype)
Docstring: <no docstring>
Source:   
def np_to_triton_dtype(np_dtype):
    if np_dtype == bool:
        return "BOOL"
    elif np_dtype == np.int8:
        return "INT8"
    elif np_dtype == np.int16:
        return "INT16"
    elif np_dtype == np.int32:
        return "INT32"
    elif np_dtype == np.int64:
        return "INT64"
    elif np_dtype == np.uint8:
        return "UINT8"
    elif np_dtype == np.uint16:
        return "UINT16"
    elif np_dtype == np.uint32:
        return "UINT32"
    elif np_dtype == np.uint64:
        return "UINT64"
    elif np_dtype == np.float16:
        return "FP16"
    elif np_dtype == np.float32:
        return "FP32"
    elif np_dtype == np.float64:
        return "FP64"
    elif np_dtype == np.object_ or np_dtype.type == np.bytes_:
        return "BYTES"
    return None
File:      /usr/local/lib/python3.8/dist-packages/tritonclient/utils/__init__.py
Type:      function


In [34]:
np_to_triton_dtype(amount_data.dtype)

In [24]:
inputs[0]

In [12]:
outputs = [
    httpclient.InferRequestedOutput("AMOUNT")
]

In [13]:
response = client.infer(model_name,
                        inputs, outputs=outputs)

InferenceServerException: Unable to parse 'datatype': attempt to access JSON non-string as string

In [ ]:
output0_data = response.as_numpy("AMOUNT")

In [ ]:
print(output0_data)

In [14]:
model_config_str = """
{"name":"preprocessing","platform":"","backend":"python","version_policy":{"latest":{"num_versions":1}},"max_batch_size":8,"input":[{"name":"AMOUNT","data_type":"TYPE_STRING","format":"FORMAT_NONE","dims":[1],"is_shape_tensor":false,"allow_ragged_batch":false,"optional":false}],"output":[{"name":"AMOUNT","data_type":"TYPE_FP32","dims":[1],"label_filename":"","is_shape_tensor":false}],"batch_input":[],"batch_output":[],"optimization":{"priority":"PRIORITY_DEFAULT","input_pinned_memory":{"enable":true},"output_pinned_memory":{"enable":true},"gather_kernel_buffer_threshold":0,"eager_batching":false},"instance_group":[{"name":"preprocessing_0","kind":"KIND_GPU","count":1,"gpus":[0],"secondary_devices":[],"profile":[],"passive":false,"host_policy":""}],"default_model_filename":"","cc_model_filenames":{},"metric_tags":{},"parameters":{},"model_warmup":[]}
"""

In [15]:
model_config = json.loads(model_config_str)

In [36]:
X_infer = pd.read_parquet("X_test.parquet")

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.